In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

**Other sources to check out:**
- https://www.air-port-codes.com/
- https://www.unipage.net/en/airport/{airport name}

### Read file and take Airport list unique values

In [2]:
import os
os.getcwd()

'g:\\OneDrive - Airports Corporation of Vietnam - ACV\\Work Data\\WORK_DATA\\Airport Info'

In [3]:
df = pd.read_csv('../SMIS/Y2015-T3Y2023.csv')

C:\Users\duyta\AppData\Local\Temp\ipykernel_10824\3121896667.py:1: DtypeWarning: Columns (8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../SMIS/Y2015-T3Y2023.csv')


In [5]:
df.head()

,FlightDate,FlightNo,ACType,Route,CgoKgs,TotalPax,BaseStation,BagPcs,ScheduledTime,FlightType,Airline,domint,Capacity,INF,CHD,ADL,ArrDep,C
0,2015-01-01,BL570,A320,SGN-BMV,0,185,BMV,26,0755,PAX,BL,True,180.0,9,18,158,A,0
1,2015-01-01,BL571,A320,BMV-SGN,0,146,BMV,10,0825,PAX,BL,True,180.0,4,19,123,D,0
2,2015-01-01,VJ352,A320,SGN-BMV,654,176,BMV,51,1620,PAX,VJ,True,180.0,5,16,155,A,0
3,2015-01-01,VJ353,A320,BMV-SGN,3713,152,BMV,32,1340,PAX,VJ,True,180.0,6,23,123,D,0
4,2015-01-01,VJ495,A320,HAN-BMV,3713,171,BMV,57,1305,PAX,VJ,True,180.0,2,10,159,A,0


In [6]:
df = df[df['Route'].str.len() == 7]

In [7]:
set1 = set(df['Route'].str[:3])
set2 = set(df['Route'].str[4:])
Airport_list = list(set(set1) | set(set2))

In [8]:
len(Airport_list)

765

### Airport Country Search

#Test cell
dct = {}

for val in Airport_list[:5]:
    url = f'https://www.travelmath.com/airport/{val}'
    r = requests.get(url)
    resp = r.content
    soup = BeautifulSoup(resp, "html.parser")
    t = soup.select('a[href^="/country/"]')
    if len(t) >0:
        for a in t:
            dct[val] = a.string
            time.sleep(0.5)
    else:
        dct[val] = "Not Available"
        
df_country_search = pd.DataFrame(list(dct.items()), columns = ['Airport Code', 'Country'])        


In [8]:
dct = {}

for val in Airport_list:
    url = f'https://www.travelmath.com/airport/{val}'
    r = requests.get(url)
    resp = r.content
    soup = BeautifulSoup(resp, "html.parser")
    t = soup.select('a[href^="/country/"]')
    if len(t) >0:
        for a in t:
            dct[val] = a.string
            time.sleep(0.5)
    else:
        dct[val] = "Not Available"

df_country_search = pd.DataFrame(list(dct.items()), columns = ['Airport Code', 'Country'])
df_country_search


,Airport Code,Country
0,PVG,China
1,ATH,Greece
2,ANC,United States
3,LTI,Mongolia
4,LHR,United Kingdom
...,...,...
760,MSJ,Japan
761,ULN,Mongolia
762,BKK,Thailand
763,AMM,Jordan


In [9]:
df_country_search

,Airport Code,Country
0,PVG,China
1,ATH,Greece
2,ANC,United States
3,LTI,Mongolia
4,LHR,United Kingdom
...,...,...
760,MSJ,Japan
761,ULN,Mongolia
762,BKK,Thailand
763,AMM,Jordan


### Airport Latitude-Longitude Search

#Test cell
lst = []
for value in Airport_list[:5]:
    url = f'https://www.travelmath.com/airport/{value}'
    r = requests.get(url)
    resp = r.content
    soup = BeautifulSoup(resp, "html.parser")
    t = soup.select("p")[2].text
    category = 'Latitude/Longitude'
    if category  in t:
        appendValue = t.split(':')[1]
        lst.append([value,appendValue])
    else:
        lst.append([value,'Not Available, Not Available'])
        
df_lat_long_search = pd.DataFrame(lst, columns = ['Airport Code', 'Lat/Long'])
df_lat_long_search[['Latitude', 'Longitude']] = df_lat_long_search['Lat/Long'].str.split(',', expand = True)
df_lat_long_search.drop(['Lat/Long'], axis = 1, inplace = True)

In [10]:
#New code(16-02-2022)
lst = []
for value in Airport_list:
    url = f'https://www.travelmath.com/airport/{value}'
    r = requests.get(url)
    resp = r.content
    soup = BeautifulSoup(resp, "html.parser")
    t = soup.select("p")[2].text
    category = 'Latitude/Longitude'
    if category  in t:
        appendValue = t.split(':')[1]
        lst.append([value,appendValue])
    else:
        lst.append([value,'Not Available, Not Available'])

df_lat_long_search = pd.DataFrame(lst, columns = ['Airport Code', 'Lat/Long'])
df_lat_long_search[['Latitude', 'Longitude']] = df_lat_long_search['Lat/Long'].str.split(',', expand = True)
df_lat_long_search.drop(['Lat/Long'], axis = 1, inplace = True)


In [11]:
df_lat_long_search

,Airport Code,Latitude,Longitude
0,PVG,31.1477777778,121.7980555556
1,ATH,37.9363888889,23.9444444444
2,ANC,Not Available,Not Available
3,LTI,Not Available,Not Available
4,LHR,51.47138888,-0.45277777
...,...,...,...
760,MSJ,40.70528,141.37194
761,ULN,47.84306,106.76639
762,BKK,13.6827777778,100.7544444444
763,AMM,31.7225,35.9933333333


### IATA Airport search

#Test cell
lst = []
val = Airport_list[:5]

url = f'https://www.iata.org/en/publications/directories/code-search/?airport.search={val}'
#print(url)
#url = f'https://flightmath.com/from-{val[:3]}-to-{val[-3:]}'
r = requests.get(url)
resp = r.content
soup = BeautifulSoup(resp, "html.parser")
#print(soup)
data = []
try:
    for tr in soup.find('table', class_='datatable').find_all('tr')[1:]:
        row = [td.text for td in tr.find_all('td')]
        #print(row)
        lst.append(row)
except:
    row = lst.append(['Not Available', 'Not Available', val])
    
df_name_search = pd.DataFrame(lst, columns=['City', 'Airport Name', 'Airport Code'])

In [12]:
lst = []
for val in Airport_list:

    url = f'https://www.iata.org/en/publications/directories/code-search/?airport.search={val}'
    r = requests.get(url)
    resp = r.content
    soup = BeautifulSoup(resp, "html.parser")
    #    print(soup)
    data = []
    try:
        for tr in soup.find('table', class_='datatable').find_all('tr')[1:]:
            row = [td.text for td in tr.find_all('td')]
            #print(row)
            lst.append(row)
    except:
        row = lst.append(['Not Available', 'Not Available', val])
df_name_search = pd.DataFrame(lst, columns=['City', 'Airport Name', 'Airport Code'])       


In [13]:
df_name_search

,City,Airport Name,Airport Code
0,Shanghai,Pudong Intl,PVG
1,Athens,Eleftherios Venizelos,ATH
2,Anchorage,Metropolitan Area,ANC
3,Altai,Altai,LTI
4,London,Heathrow,LHR
...,...,...,...
873,Misawa,Misawa,MSJ
874,Ulaanbaatar,Buyant-Ukhaa,ULN
875,Bangkok,Suvarnabhumi Int'l,BKK
876,Amman,Queen Alia Intl,AMM


### <span style="color:#DC143C">**New Info website: unipage.net**</span>

#Test code
lst = []
lst2 = []
url = f'https://www.unipage.net/en/airport/CSR'
r = requests.get(url)
resp = r.content
soup = BeautifulSoup(resp, "html.parser")

x = soup.findAll(class_ = "infographic-card__title")
for p in x:
    
    if p.string == 'Country' \
        or p.string == 'Latitude' \
        or p.string == 'Longitude' \
        or p.string == 'ACII'\
        or p.string == 'Region':
        t = p.parent
        for child in t:

            if child.string != None:
                #print(child)
                lst2.append(child.string)
        
        #print('----------------')

dct  = dict(zip(lst2[::2], lst2[1::2]))
lst.append(dct)
print(lst)
pd.DataFrame(lst)


In [110]:
lst = []

for val in Airport_list:
    dct = {}
    lst2 = []
    url = f'https://www.unipage.net/en/airport/{val}'
    r = requests.get(url)
    resp = r.content
    soup = BeautifulSoup(resp, "html.parser")

    x = soup.findAll(class_ = "infographic-card__title")
    for p in x: 
        if p.string == 'Country' \
            or p.string == 'Latitude' \
            or p.string == 'Longitude' \
            or p.string == 'ACII'\
            or p.string == 'Region':
            t = p.parent
            for child in t:

                if child.string != None:
                    #print(child)
                    lst2.append(child.string)   
    dct  = dict(zip(lst2[::2], lst2[1::2]))
    dct['Airport Code'] = val
    
    lst.append(dct)

In [111]:
df_unipage = pd.DataFrame(lst)
df_unipage = df_unipage[['Airport Code', 'ACII', 'Country', 'Region', 'Latitude', 'Longitude']]
df_unipage.fillna('Not Available', inplace = True)
df_unipage.rename(columns = {'ACII': 'Airport Name'}, inplace = True)

In [ ]:
df_unipage

In [ ]:
from datetime import date
t = str(date.today()).replace('-', '')

In [ ]:
df_unipage.to_excel(f'{t}-AirportInfo(2015-2022)-UniPage.xlsx', index = False)

### Combine 3 DataFrames

In [14]:
df_all = pd.merge(df_country_search,pd.merge(df_name_search,df_lat_long_search,how = 'inner'),how = 'inner')

In [15]:
df_all.head()

,Airport Code,Country,City,Airport Name,Latitude,Longitude
0,PVG,China,Shanghai,Pudong Intl,31.1477777778,121.7980555556
1,ATH,Greece,Athens,Eleftherios Venizelos,37.9363888889,23.9444444444
2,ANC,United States,Anchorage,Metropolitan Area,Not Available,Not Available
3,LTI,Mongolia,Altai,Altai,Not Available,Not Available
4,LHR,United Kingdom,London,Heathrow,51.47138888,-0.45277777


In [16]:
from datetime import date
t = str(date.today()).replace('-', '')

In [17]:
df_all.to_excel(f'{t}-AirportInfo(2015-2022).xlsx', index = False)